In [1]:
driver.close()

NameError: name 'driver' is not defined

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options as FirefoxOptions

# Initialize Firefox Options
firefox_options = FirefoxOptions()

# Create a new Firefox Profile for your extensions
firefox_profile = webdriver.FirefoxProfile()

# Convert the profile to a base64 string and add it to the options
firefox_options.profile = firefox_profile

In [3]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allensunny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Set up the service with GeckoDriverManager
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.install_addon('/Users/allensunny/Downloads/Mozilla Scraper Tools/uBlock0_1.55.0.firefox.signed.xpi',  temporary=True)
driver.install_addon('/Users/allensunny/Downloads/Mozilla Scraper Tools/bypass_paywalls_clean-3.5.5.0.xpi', temporary=True)

'magnolia@12.34'

In [5]:
#Webscrpaing and request parsing 
import requests
from bs4 import BeautifulSoup, Comment
import time

from lxml import etree
from lxml.html import fromstring

In [6]:
from googlesearch import search

In [7]:
#Text summarizer intialization 
from transformers import pipeline
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

/Users/allensunny/anaconda3/envs/oxford_new/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Setting the toeknizer parallizsm explicityl 
import os
# Enable parallelism in tokenizers for improved performance
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [9]:
import time
import re

In [10]:
#Druid database dump
import pydruid

In [11]:
#Basic packages installation
import pandas as pd
import numpy as np

/var/folders/jq/vkbby4bd33xbf6kks2_zrswh0000gn/T/ipykernel_96901/3073400958.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
import random
import os

In [13]:
#Proxy Function extraction 
def get_random_proxy():
    return random.choice(proxies)

In [14]:
#Consutructing the proxy farm 
with open('/Users/allensunny/Downloads/http.txt', 'r') as file:
    content = file.read()

#Converting to a list of proxies to fetch data
proxies = content.strip().split('\n')

In [15]:
#Getting the list of companies
with open('/Users/allensunny/Downloads/Company_list.txt', 'r') as file:
    content = file.read()

#Cleaning the company names 
def clean_company_name(name):
    
    # Remove common corporate designations
    cleaned_name = re.sub(r'\s*(Inc\.|Co\.|Corp\.|Corporation|Company|Incorporated|LLC|Group|Corp|Enterprises|International|Co\.\,|L\.P\.|Holdings|Inc|Co)\.?$', '', name, flags=re.IGNORECASE)
    return cleaned_name.strip()

# Split the cleaned text into a list based on multiple spaces as a separator
company_list = content.split('    ')  # Adjust the number of spaces based on how the original text is structured
company_list = [company.strip() for company in company_list if company.strip()]

# Further cleaning to ensure no leading/trailing spaces in company names
company_list = [company.strip() for company in company_list]

# Apply the cleaning function to each company name
cleaned_companies = [clean_company_name(company) for company in company_list]

In [16]:
#Opening each tab and extracting the text within it 
#Companies = ['Nike']
target_phrases = ["Generative AI", "LLM", "ChatGPT", "Llama-2", "BLOOMZ", "GPT-4", "Stable Diffusion 2", "PaLM 2", "Claude 2", "Jurrasic-2", "Inflection-1", "Titan Text"]


#Note that this is an issue where we cant extract the company name for the data set we have provided, probably we can do 
#Some more clever keyword extraction rather than idntifying every word 
#target_phrases = target_phrases + cleaned_companies

In [36]:
cleaned_companies[11]

'Administaff,'

In [44]:
#Testing
"""
Fetch web pages related to the company's AI usage and print a summary of the content.

:param company_name: The name of the company to search for.
"""
results = []  # List to store result dictionaries
company_name = "Quora"

#Create a randomization function for this 

#I need a time out query that cuts the execution if the the code runs above a certain time 

#Generate a random proxy 
proxy = get_random_proxy()

#Call the search query
search_query = f"{company_name} AI Usage In within the company"
search_results = search(search_query, num_results = 10, proxy = proxy, timeout = 10)  

for url in search_results:
    
    print(url)
    try:
        # Set a timeout for the request
        response = requests.get(url, timeout=10)
        
        # Process your response here
        print(response.status_code)
        print("Code has fetched the url")

    #Note that the timeout function isnt working for some reason, I'll have implement the multithreading logic 
    except requests.exceptions.Timeout:
        print(f"Request timed out for {url}")

    #Handles all other exceptions
    except requests.exceptions.RequestException as e:
        
        # Handles other requests exceptions
        print(f"Request error for {url}: {e}")
 
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            print("Rate limit exceeded, waiting before retrying...")
            time.sleep(10)
            print('Slept')

    if response.status_code == 200:
        print('In the 200 stautus code situation')
        page_source = response.text
        soup = BeautifulSoup(page_source, 'html.parser')

        #Inital cleaning
        for a_tag in soup.find_all('a'):
            a_tag.decompose()  # This removes the tag and its content
            
        # Extract text from the modified BeautifulSoup object
        text = soup.get_text(strip=True)
        
        print('About to summarize using t5')
        
        #Calling the summarization function
        summarized_text = summarize_and_extract(text, target_phrases, summarizer, max_length=1000, min_length=5, do_sample=False)
        print('Done summary')

        
        # Append result to the list
        results.append({'URL': url, 'Summary': summarized_text, 'Company': company_name})

    
    else:
        print(f"Failed to retrieve {url}: Status code {response.status_code}")
        
        #Here I call the selium function to run it 
        #Initalize the selinium value
        print('Starting selinium')
        
        driver.get(url)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        
        #Inital cleaning
        for a_tag in soup.find_all('a'):
            a_tag.decompose()  # This removes the tag and its content
            
        # Extract text from the modified BeautifulSoup object
        text = soup.get_text(strip=True)

        #Calling the summarization function
        summarized_text = summarize_and_extract(text, target_phrases, summarizer, max_length=1000, min_length=5, do_sample=False)
        
        # Append result to the list
        results.append({'URL': url, 'Summary': summarized_text, 'Company': company_name})
        
    
    # Convert the list of dictionaries to a DataFrame
    results_df = pd.DataFrame(results)

results_df
#return results_df

https://www.quora.com/Do-the-majority-of-companies-who-claim-to-be-using-AI-actually-do


Your max_length is set to 1000, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


200
Code has fetched the url
In the 200 stautus code situation
About to summarize using t5
Done summary
https://www.quora.com/What-is-the-role-of-AI-in-business


Your max_length is set to 1000, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


200
Code has fetched the url
In the 200 stautus code situation
About to summarize using t5
Done summary
https://www.quora.com/What-will-be-the-impact-of-the-role-of-artificial-intelligence-in-business


...

Your max_length is set to 1000, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


200
Code has fetched the url
In the 200 stautus code situation
About to summarize using t5
Done summary
https://www.quora.com/Can-AI-help-us-to-build-a-multi-billion-dollar-company


Your max_length is set to 1000, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


200
Code has fetched the url
In the 200 stautus code situation
About to summarize using t5
Done summary


,URL,Summary,Company
0,https://www.quora.com/Do-the-majority-of-compa...,. .,Quora
1,https://www.quora.com/What-is-the-role-of-AI-i...,. .,Quora
2,https://www.quora.com/What-will-be-the-impact-...,. .,Quora
3,https://www.quora.com/How-do-businesses-use-ar...,. .,Quora
4,https://www.quora.com/How-is-artificial-intell...,. .,Quora
5,https://www.quora.com/Why-would-a-company-want...,. .,Quora
6,https://www.quora.com/Can-you-give-some-exampl...,. .,Quora
7,https://www.quora.com/What-percentage-of-busin...,. .,Quora
8,https://www.quora.com/What-are-the-benefits-of...,. .,Quora
9,https://www.quora.com/Is-it-legal-to-use-artif...,. .,Quora


In [37]:
Results = fetch_and_summarize_company_ai_usage(cleaned_companies[12])
cleaned_companies[12]

https://www.cnet.com/tech/computing/adobe-says-its-new-generative-ai-will-help-creative-pros-not-hurt-them/
https://www.techrepublic.com/article/gpt-4-cheat-sheet/
https://elearning.adobe.com/2023/08/using-ai-to-create-an-interactive-scenario-with-the-all-new-adobe-captivate-part-one/
https://community.openai.com/t/gpt-4-api-for-educational-application/571083
https://fstoppers.com/artificial-intelligence/how-chatgpt-4-saved-me-hours-after-effects-last-night-632360
https://www.toolify.ai/ai-news/automate-adobe-illustrator-with-gpt4-765573


'Adobe Systems'

In [38]:
final_df = pd.DataFrame()

In [39]:
#Concating to the final output 
final_df = pd.concat([Results, final_df], axis = 0)
final_df.reset_index(inplace =True, drop = True)

In [40]:
final_df.to_csv('Companies_final_2.csv', index=False)

In [32]:
#Function to take a funciton and summarize the AI usage 
def fetch_and_summarize_company_ai_usage(company_name):
    """
    Fetch web pages related to the company's AI usage and print a summary of the content.
    
    :param company_name: The name of the company to search for.
    """
    results = []  # List to store result dictionaries
    #Generate a random proxy 
    proxy = get_random_proxy()
    
    search_query = f"{company_name} Usage of GPT-4"
    search_results = search(search_query, num_results = 5, proxy = proxy, timeout = 10)  
                
    for url in search_results:
        if url.endswith('.pdf'):
            continue  # Skip PDF files  
        
        try:
            response = requests.get(url)
            response.raise_for_status()  # This will directly raise an exception for HTTP errors
            page_source = response.text
            
            print(url)

        #Raising the selinium exception logic
        except requests.exceptions.RequestException:
            print(f"Failed to retrieve {url}: Status code {response.status_code}")
            
            #Here I call the selium function to run it 
            driver.get(url)
            page_source = driver.page_source
            page_source = response.text

            print(url)
            
        #Regular runtime with the requests logic libary 
        page_source = response.text
        soup = BeautifulSoup(page_source, 'html.parser')

        #Inital cleaning
        for a_tag in soup.find_all('a'):
            a_tag.decompose()  # This removes the tag and its content
            
        # Extract text from the modified BeautifulSoup object
        text = soup.get_text(strip=True)

        #Calling the summarization function
        summarized_text = summarize_and_extract(text, target_phrases, summarizer, max_length=50, min_length=5, do_sample=False)
        
        # Append result to the list
        results.append({'URL': url, 'Summary': summarized_text, 'Company': company_name})
    
        results_df = pd.DataFrame(results)
    
    return results_df        

In [22]:
#Text keyword summarizer
def summarize_and_extract(text, target_phrases, summarizer, max_length=100, min_length=5, do_sample=True):
    """
    Summarize the given text and then extract sentences containing any of the target phrases.
    
    :param text: The text to summarize and extract sentences from.
    :param target_phrases: A list of phrases to look for in the summarized text.
    :param summarizer: The summarization function to use.
    :param max_length: The maximum length of the summary.
    :param min_length: The minimum length of the summary.
    :param do_sample: Whether or not to sample in the summarization process.
    :return: A string containing the extracted sentences from the summarized text.
    """
    #Perform the text preprocessing 
    find_text = find_sentences_with_keywords(text, target_phrases)

    # Join extracted sentences into a cohesive summary
    extracted_summary = " ".join(find_text)

    # Perform summarization
    summarized = summarizer(extracted_summary, max_length=max_length, min_length=min_length, do_sample=do_sample, early_stopping=True)
    summarized_text = summarized[0]['summary_text']  # Assuming the summarizer returns a list of dictionariess

    return summarized_text

In [23]:
#I like how this system takes the sentences, but Iwould like to improve the process by hitting the previous and next sentences to give the system more
#Context 

#Key word extraction function 
def find_sentences_with_keywords(sentence_text, keywords):
    
    # Tokenize the text into sentences
    sentences = sent_tokenize(sentence_text)
    
    keyword_sentences = []
    
    # Loop through sentences
    for sentence in sentences:
        
        # Find all keywords present as whole words in the sentence
        found_keywords = [keyword for keyword in keywords if re.search(r'\b' + re.escape(keyword) + r'\b', sentence, re.IGNORECASE)]
        
        if found_keywords:
            # Append sentence and found keywords to the list
            keyword_sentences.append(sentence)
    
    return keyword_sentences

In [24]:
#Function to take a funciton and summarize the AI usage 
def Foundation_model_partners(company_name, products):
    """
    Fetch web pages related to the company's AI usage and print a summary of the content.
    
    :param company_name: The name of the company to search for.
    """
    results = []  # List to store result dictionaries

    #Generate a random proxy 
    proxy = get_random_proxy()
    
    search_query = f"Users or companies or partners or collaborators of {company_name} {products} list"
    search_results = search(search_query, num_results = 5, proxy = proxy, timeout = 10)  
        
    for url in search_results:
        try:
            response = requests.get(url)
            response.raise_for_status()  # This will directly raise an exception for HTTP errors
            page_source = response.text

            print(url)

        #Raising the selinium exception logic
        except requests.exceptions.RequestException:
            print(f"Failed to retrieve {url}: Status code {response.status_code}")
            
            #Here I call the selium function to run it 
            driver.get(url)
            page_source = driver.page_source
            page_source = response.text

            print(url)
            
        #Regular runtime with the requests logic libary 
        page_source = response.text
        soup = BeautifulSoup(page_source, 'html.parser')

        #Inital cleaning
        for a_tag in soup.find_all('a'):
            a_tag.decompose()  # This removes the tag and its content
            
        # Extract text from the modified BeautifulSoup object
        text = soup.get_text(strip=True)

        #Calling the summarization function
        summarized_text = summarize_and_extract(text, target_phrases, summarizer, max_length=50, min_length=5, do_sample=False)
        
        # Append result to the list
        results.append({'URL': url, 'Summary': summarized_text, 'Company': company_name})
    
        results_df = pd.DataFrame(results)
    
    return results_df    

In [22]:
company_name = "Stability AI"
products = "Stable Diffusion 2"

In [23]:
results_df = Foundation_model_partners(company_name, products)

Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


https://stability.ai/news/stable-diffusion-v2-release
https://github.com/Stability-AI/stablediffusion
https://www.forbes.com/sites/kenrickcai/2023/06/04/stable-diffusion-emad-mostaque-stability-ai-exaggeration/
https://www.assemblyai.com/blog/stable-diffusion-1-vs-2-what-you-need-to-know/
Failed to retrieve https://www.cbinsights.com/investor/stability-ai: Status code 403


Your max_length is set to 50, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


https://www.cbinsights.com/investor/stability-ai
https://techcrunch.com/2023/07/26/stability-ai-releases-its-latest-image-generating-model-stable-diffusion-xl-1-0/


Your max_length is set to 50, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


https://stability.ai/partners


In [89]:
results_df['Summary'][0]

'Stable Diffusion 2.0 Release24 NovWritten ByWe are pleased to announce the open-source release of.The originalled bychanged the nature of open source AI models and spawned hundreds of other models and innovations worldwide'

In [90]:
results_df['URL'][0]

'https://stability.ai/news/stable-diffusion-v2-release'

In [91]:
#final_df = pd.DataFrame()

In [92]:
#Concating to the final output 
final_df = pd.concat([results_df, final_df], axis = 0)
final_df.reset_index(inplace =True, drop = True)

In [93]:
#Writing the final data out
final_df.to_csv('output_final_new_2.csv', index=False)